# System with spring

## Table of Contents

- [Batches and the Data Structure](#batches-and-the-data-structure)
- [Query Data](#query-data)
- [Initial Parameters](#initial-parameters)
- [System Motion Animation](#system-motion-animation)
- [Plot Trajectories](#plot-trajectories)
- [Statistics](#statistics)
- [Standard Deviation Test](#standard-deviation-test)

## Batches and the Data Structure

Let's assume we have created a batch named 'spring_system_angles' consisting of 5 simulations, in which the initial angles of the second pendulum were randomly selected from a normal distribution with a mean of 10 degrees and a standard deviation of 5 degrees.

In [ ]:
from citros_data_analysis import data_access as da

Let's create CitrosDB object to query and plot the results of the simulation. Let's specify that we would like to see batches that were created with system with spring simulation scenario:

In [ ]:
citros = da.CitrosDB(simulation = 'simulation_system_with_spring')

To print information about all batches created in 'simulation_double_pendulum' simulation, call `search_batch()`:

In [ ]:
citros.search_batch()

Let's display the names of the batches that contain simulations marked as 'DONE', signifying their successful completion:

In [ ]:
list(citros.search_batch(sid_status='DONE').keys())

Print general information about the most recent simulation. In this simulation we have two topics: '/config' and '/coordinates':

In [ ]:
citros.batch(-1).info().print()

Topic '/config' contains initial parameters of the simulation, topic '/coordinates' contains result of the simulation. Let's look on data structure of the topic '/coordinates':

In [ ]:
citros.batch(-1).topic('/coordinates').info().print()

As we stated in the README, the result of the simulation has the following structure: there is time coordinate 'data.t', two coordinates of the three pendulums: data.p1.x, data.p1.y, data.p2.x, data.p2.y, data.p3.x, data.p3.y and coordinates of the start and end of the spring: data.spr.x0, data.spr.x1, data.spr.y0, data.spr.y1:

In [ ]:
citros.batch(-1).topic('/coordinates').info()['topics']['/coordinates']['data_structure']['data'].print()

## Query Data

Let's query data by `data()` method. If we call `data()` method without arguments we get all data separated by columns.


The output of the `data()` method is a pandas.DataFrame, so every method of the pandas.DataFrame can be applied to the result of the query.
Here by `head()` method we left only first 5 rows of the output:

In [ ]:
citros.batch(-1).topic('/coordinates').data().head(5)

We can query not all data, but, for example, only time and x coordinates of the pendulums:

In [ ]:
citros.batch(-1).topic('/coordinates').data(['data.t', 'data.p1.x', 'data.p2.x', 'data.p3.x']).head(5)

## Initial Parameters

In '/config' topic we can find all the initial parameters.

In [ ]:
name = 'data.system_with_spring.ros__parameters'
params = citros.batch(-1).topic('/config').data(name, additional_columns='sid')
params = params[params[name].notna()].set_index('sid')[name]

print(f"pandas.Series with initial parameters for the simulation runs:\n{params}")

In the batch that we prepared earlier we varied the initial angle of the second pendulum picking each value from the normal distribution with mean equals 10 and standard deviation equals 5. Let's print these initial values too:

In [ ]:
name = 'data.system_with_spring.ros__parameters.a2_0'
a2_0 = citros.batch(-1).topic('/config').data(name, additional_columns='sid').rename({name: 'a2_0'}, axis = 1)
a2_0 = a2_0[a2_0['a2_0'].notna()]
print(f"\ninitial angles for the second pendulum:\n{a2_0['a2_0']}")

## System Motion Animation

Let's query data and plot the animation of the system motion, for example for the first simulation run (sid = 0). We will also need to know the distance x0 between the double and third pendulums - we can take it from the `params`, that we get earlier:

In [ ]:
F = citros.batch(-1).sid(0).topic('/coordinates').data()
x0 = params[0]['x0']

In [ ]:
import matplotlib.pyplot as plt
from collections import deque

def animate(F, x0):
    # length of the history trace
    trace_len = 100

    max_y_0 = abs(min([min(F['data.p2.y']), min(F['data.p3.y'])]))*1.2
    max_y_1 = max([max(F['data.p1.y']), max(F['data.p2.y']), max(F['data.p3.y']), 0.1])*1.2
    max_x = max([max(abs(F['data.p1.x'])), max(abs(F['data.p2.x'])), max(abs(F['data.p3.x']))])*1.2

    fig = plt.figure()
    ax = fig.add_subplot(111, autoscale_on=False, xlim=(-max_x, max_x), ylim=(-max_y_0, max_y_1))
    ax.set_aspect('equal', 'datalim')
    ax.grid()

    line1, = ax.plot([], [], 'o-', lw=2)
    line2, = ax.plot([], [], 'o-', lw=2)
    line3, = ax.plot([], [], 'o-', lw=2)
    line4, = ax.plot([], [], 'm--', lw=1)
    trace, = ax.plot([], [], 'b-', lw=0.5, ms=1)
    time_text = ax.text(0.02, 0.90, '', transform=ax.transAxes)
    trace_x, trace_y = deque(maxlen=trace_len), deque(maxlen=trace_len)

    plt.close()
    def init():
        line1.set_data([], [])
        line2.set_data([], [])
        trace.set_data([], [])
        line3.set_data([], [])
        line4.set_data([], [])
        time_text.set_text('')
        trace_x.clear()
        trace_y.clear()
        return line1, line2, trace, time_text

    def animate_frame(i):
        trace_x.appendleft(F['data.p2.x'].iloc[i])
        trace_y.appendleft(F['data.p2.y'].iloc[i])

        line1.set_data([[F['data.p1.x'].iloc[i],F['data.p2.x'].iloc[i]],[F['data.p1.y'].iloc[i], F['data.p2.y'].iloc[i]]])
        line2.set_data([[x0,F['data.p1.x'].iloc[i]],[0, F['data.p1.y'].iloc[i]]])
        line3.set_data([[0,F['data.p3.x'].iloc[i]],[0, F['data.p3.y'].iloc[i]]])
        line4.set_data([[F['data.spr.x0'].iloc[i],F['data.spr.x1'].iloc[i]],[F['data.spr.y0'].iloc[i], F['data.spr.y1'].iloc[i]]])
        trace.set_data(trace_x, trace_y)

        time_text.set_text('Time = %.3f s' % F['data.t'].iloc[i])
        return line1, line2, line3, line4, trace, time_text

    init()
    for i in range(len(F)):
        animate_frame(i)
        display(fig, clear=True)

animate(F, x0)

## Plot Trajectories

Small variations in the initial parameters of the double pendulum system lead to significant changes in trajectories.
Let's plot the trajectories for simulations where the initial angle of the second pendulum is set randomly from the normal distribution (mean = 10, std = 5)

Let's print the initial angles using the '/config' topic:

In [ ]:
col_name = 'data.system_with_spring.ros__parameters.a2_0'

a2_0 = citros.batch(-1).topic('/config').data(col_name, additional_columns='sid').rename({col_name: 'a2_0'}, axis = 1)
a2_0 = a2_0 [a2_0 ['a2_0'].notna()].set_index('sid')
a2_0

Let's plot the trajectory of the second pendulum for the first 5 seconds of the simulations:

In [ ]:
F = citros.batch(-1).set_filter({'data.t': {'<': 5}}).topic('/coordinates').data()
citros.plot_graph(F, 'data.p2.x', 'data.p2.y')

Let's plot the trajectory of the second pendulum from the first and second simulations:

In [ ]:
F = citros.batch(-1).set_filter({'data.t': {'<': 5}}).topic('/coordinates').sid([0,1]).data()
citros.plot_graph(F, 'data.p2.x', 'data.p2.y')

## Statistics

We can examine how the coordinates of the second pendulum, along with their mean values and standard deviations, change over time using the `citros_data_analysis.error_analysis` package:

In [ ]:
from citros_data_analysis import error_analysis as analysis
F = citros.batch(-1).topic('/coordinates').data()
dataset = analysis.CitrosData(F, data_label=['data.p2.x', 'data.p2.y'], units = 'm')
db = dataset.bin_data(n_bins = 50, param_label = 'data.t')
db.show_statistics()

Let's plot the scatter plot of the last points of the simulation (around 'data.t' = 10). Let's also depict 1-, 2-, 3-sigma error ellipses and print parameters of the 1-sigma ellipse:

In [ ]:
ellipse_param = db.show_correlation(x_col = 'data.p2.x', y_col = 'data.p2.y',
                      slice_val = 10, n_std = [1,2,3], return_ellipse_param = True)
ellipse_param[0]

## Standard Deviation Test

By utilizing different tests from the `citros_data_analysis.validation` package, you can quickly determine whether the simulation meets specific conditions. For instance, let's check if the 3-sigma standard deviation does not exceed 0.15 for x coordinate and less then 0.1 for y coordinate:

In [ ]:
from citros_data_analysis import validation as va
F = citros.batch(-1).topic('/coordinates').data()
V = va.Validation(F, data_label = ['data.p2.x', 'data.p2.y'], param_label = 'data.t', method = 'bin', num = 50, units = 'm')
log, table, fig = V.std_test(limits = [0.15, 0.1], n_std = 3, std_area = True)

In [ ]:
ref = da.Ref()
ref.print()